1 - Import Relevant Libraries

In [1]:
import numpy as np
import tensorflow as tf

2 - Load Dataset

In [2]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype("float")
train_targets = npz['targets'].astype("int")

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype("float"), npz['targets'].astype("int")

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype("float"), npz['targets'].astype("int")

3 - Model, Optimizer and Loss Function

In [3]:
output_size = 2
hidden_layer_size = 50

# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(
        hidden_layer_size, activation='relu'),  # 1st hidden layer
    tf.keras.layers.Dense(
        hidden_layer_size, activation='relu'),  # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax')  # output layer
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

4 - Batching, Early Stopping, Training and Validating

In [4]:
# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
# patience lets us decide how many consecutive increases in val_accuracy we can tolerate
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs,  # train inputs
            train_targets,  # train targets
            batch_size=batch_size,  # batch size
            # epochs that we will train for (assuming early stopping doesn't kick in)
            epochs=max_epochs,
            # callbacks are functions called by a task when a task is completed
            # task here is to check if val_loss is increasing
            callbacks=[early_stopping],  # early stopping
            # validation data
            validation_data=(validation_inputs, validation_targets),
            verbose=2  # making sure we get enough information about the training process
            )

Epoch 1/100
36/36 - 2s - loss: 0.6210 - accuracy: 0.6628 - val_loss: 0.5352 - val_accuracy: 0.7204 - 2s/epoch - 44ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4926 - accuracy: 0.7469 - val_loss: 0.4552 - val_accuracy: 0.7494 - 219ms/epoch - 6ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4354 - accuracy: 0.7692 - val_loss: 0.4069 - val_accuracy: 0.7718 - 199ms/epoch - 6ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4055 - accuracy: 0.7840 - val_loss: 0.3854 - val_accuracy: 0.7897 - 194ms/epoch - 5ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3856 - accuracy: 0.7966 - val_loss: 0.3717 - val_accuracy: 0.7964 - 188ms/epoch - 5ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3741 - accuracy: 0.7946 - val_loss: 0.3604 - val_accuracy: 0.8098 - 189ms/epoch - 5ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3679 - accuracy: 0.8011 - val_loss: 0.3572 - val_accuracy: 0.8076 - 186ms/epoch - 5ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3624 - accuracy: 0.8103 - val_loss: 0.3513 - val_accuracy: 0.8031 - 183ms/epoch - 5ms/step
Epoch 9/10

5 - Testing

In [7]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 4ms/step - loss: 0.2963 - accuracy: 0.8393


In [8]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(
    test_loss, test_accuracy*100.))


Test loss: 0.30. Test accuracy: 83.93%


6 - Customer Conversion Probability 

In [10]:
# We can predict the probability of each class using the 'predict' method
model.predict(test_inputs)
# [prob_not_converting, prob_converting]

14/14 [==============================] - 0s 3ms/step


array([[1.77110149e-08, 1.00000000e+00],
       [9.99999881e-01, 7.53409992e-08],
       [9.99535322e-01, 4.64667071e-04],
       [6.35729611e-01, 3.64270389e-01],
       [4.66620952e-01, 5.33379018e-01],
       [9.99024391e-01, 9.75654402e-04],
       [5.64085960e-01, 4.35914040e-01],
       [9.99854922e-01, 1.45045517e-04],
       [4.82595503e-01, 5.17404437e-01],
       [2.05392107e-01, 7.94607878e-01],
       [4.71365929e-01, 5.28634071e-01],
       [9.99996066e-01, 3.98892098e-06],
       [4.79989111e-01, 5.20010829e-01],
       [1.00000000e+00, 4.25744051e-10],
       [3.97250146e-01, 6.02749944e-01],
       [4.22568977e-01, 5.77431083e-01],
       [6.95176840e-01, 3.04823130e-01],
       [9.99658108e-01, 3.41876643e-04],
       [5.47506154e-01, 4.52493817e-01],
       [9.99961138e-01, 3.88826193e-05],
       [4.84976858e-01, 5.15023172e-01],
       [5.58142304e-01, 4.41857725e-01],
       [2.13670824e-02, 9.78632867e-01],
       [6.95176840e-01, 3.04823130e-01],
       [4.825955

In [13]:
# Alternatively, we can get only the second column
# The main idea is that we are often interested in ONLY ONE of the two outcomes
# In this case we would like to know if the customer will convert again
# Once more, we can round to 0 digits, to achieve only 0s or 1s
model.predict(test_inputs)[:,1].round(0).astype("int")

14/14 [==============================] - 0s 3ms/step


array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,

In [15]:
# A much better approach here would be to use argmax (arguments of the maxima)
# Argmax indicates the position of the highest argument row-wise or column-wise
# In our case, we want ot know which COLUMN has the higher argument (probability), therefore we set axis=1 (for columns)
# The output would be the column ID with the highest argument for each observation (row)
# For instance, the first observation (in our output) was [0.93,0.07]
# np.argmax([0.93,0.07], axis=1) would find that 0.91 is the higher argument (higher probability) and return 0
# This method is great for multi-class problems as it is independent of the number of classes
predicted_test_targets = np.argmax(model.predict(test_inputs),axis=1)
predicted_test_targets

14/14 [==============================] - 0s 3ms/step


array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,

7 - Save Model

In [ ]:
# Finally we save the model using the built-in method TensorFlow method
# We choose the name and the file extension
# Since the HDF format is optimal for large numerical objects, that's our preferred choice here (and the one recommended by TF)
# The proper extension is .h5 to indicate HDF, version 5
model.save('audiobooks_model.h5')